RollingBall:玩家（或智能体）通过施加力控制小球，从网格起点（[0.2*宽, 0.2*高]）移动到目标点（[0.8*宽, 0.8*高]）。

核心特点：
- **环境**：10x10 网格（可调），小球受力、摩擦（系数 0.0046）、边界碰撞（恢复系数 0.8）影响，速度上限 5.0。
- **动作**：连续力（[-0.1, 0.1]），可离散化为 5x5 动作（[-0.8, -0.4, 0, 0.4, 0.8]）并展平为 25 个动作。
- **奖励**：每步 -2.0，撞墙 -10.0，到达目标 +300.0。
- **渲染**：Pygame 显示蓝色小球、紫色目标，可选灰色轨迹，300x300 像素窗口。

In [ ]:
import numpy as np
import random
import torch
import matplotlib.pyplot as plt
from tqdm import tqdm
import os
from gym.utils.env_checker import check_env
from RL_DQN_Class import DQN,ReplayBuffer
from gym.wrappers import TimeLimit
from two_dimensional_rolling_motion import RollingBall, DiscreteActionWrapper, FlattenActionSpaceWrapper

In [ ]:
# 设置 PyTorch 默认浮点类型为 torch.float32，确保张量数据类型一致，防止类型不匹配错误
torch.set_default_dtype(torch.float32)

# 定义移动平均函数，用于平滑回报曲线，便于观察训练趋势
def moving_average(a, window_size):
    """
    计算数组 a 的移动平均，用于平滑数据
    Args:
        a (array-like): 输入数据（如回报列表）
        window_size (int): 移动平均窗口大小
    Returns:
        np.ndarray: 平滑后的数据，长度与输入相同
    """
    result = np.convolve(a, np.ones(window_size)/window_size, mode='valid')
    pad_before = (window_size - 1) // 2
    pad_after = window_size - 1 - pad_before
    return np.pad(result, (pad_before, pad_after), mode='edge')

# 定义设置随机种子的函数，确保实验可重复
def set_seed(env, seed=42):
    """
    设置环境和全局随机种子，确保训练结果可复现
    Args:
        env: Gym 环境对象
        seed (int): 随机种子，默认为 42
    """
    env.action_space.seed(seed)
    env.reset(seed=seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)

if __name__ == "__main__":
    # 定义超参数
    state_dim = 4  # 状态维度（x位置, y位置, x速度, y速度）
    action_bins = 10  # 动作空间离散化粒度（每个维度10个离散动作）
    action_range = action_bins * action_bins  # 总动作数（10x10=100）
    action_dim = action_range  # 动作维度，与 action_range 一致
    hidden_dim = 32  # 神经网络隐藏层维度
    lr = 1e-3  # 学习率
    num_episodes = 1000  # 总训练回合数
    gamma = 0.99  # 折扣因子，决定未来奖励的重要性
    epsilon_start = 0.1  # 初始 epsilon 值，用于 ε-greedy 策略的探索
    epsilon_end = 0.001  # 最终 epsilon 值，探索率衰减的目标
    buffer_size = 10000  # 经验回放池最大容量
    minimal_size = 5000  # 开始训练前回放池的最小样本数
    batch_size = 128  # 每次训练的批量大小
    device = torch.device("cuda") if torch.cuda.is_available() else torch.device("cpu")  # 选择 GPU 或 CPU

    # 构建环境
    env = RollingBall(width=5, height=5, show_epi=True)  # 创建 RollingBall 环境，5x5 网格，显示轨迹
    env = FlattenActionSpaceWrapper(DiscreteActionWrapper(env, bins=10))  # 离散化动作空间并展平为 1D
    env = TimeLimit(env, 100)  # 设置最大步数限制为 100 步
    check_env(env.unwrapped)  # 验证环境是否符合 Gym 规范
    set_seed(env, seed=42)  # 设置随机种子

    # 验证环境输出
    state, _ = env.reset()  # 重置环境，获取初始状态
    print(f"Initial state: {state}, dtype: {state.dtype}")  # 打印初始状态和数据类型，确保为 np.float32

    # 构建智能体
    replay_buffer = ReplayBuffer(buffer_size)  # 创建经验回放池
    agent = DQN(state_dim, hidden_dim, action_dim, action_range, lr, gamma, epsilon_start, device)  # 创建 DQN 智能体
    print(f"Q_net weight dtype: {next(agent.Qnet.parameters()).dtype}")  # 打印 Q 网络参数类型，确保为 torch.float32

    # 填充经验回放池
    state, _ = env.reset()  # 重置环境
    while replay_buffer.len_buffer() <= minimal_size:
        action = env.action_space.sample()  # 随机采样动作
        next_state, reward, terminated, truncated, info = env.step(action)  # 执行动作
        replay_buffer.push_transition((state, action, reward, next_state, terminated or truncated))  # 存储经验
        state = next_state
        if terminated or truncated:
            state, _ = env.reset()  # 如果回合结束，重置环境
    print(f"Replay buffer filled with {replay_buffer.len_buffer()} transitions")  # 打印回放池填充状态

    # 训练
    return_list = []  # 存储每个回合的总回报
    max_q_value_list = []  # 存储最大 Q 值的历史
    max_q_value = 0  # 初始化最大 Q 值
    epsilon_decay = (epsilon_start - epsilon_end) / num_episodes  # 计算 epsilon 线性衰减率
    os.makedirs('./result', exist_ok=True)  # 创建结果保存目录

    for i in range(20):  # 分 20 次迭代，每迭代训练 num_episodes/20 回合
        with tqdm(total=int(num_episodes / 20), desc='Iteration %d' % i) as pbar:  # 使用 tqdm 显示进度条
            for i_episode in range(int(num_episodes / 20)):
                # 更新 epsilon 值，线性衰减以减少探索
                agent.epsilon = max(epsilon_end, epsilon_start - (i * num_episodes / 20 + i_episode) * epsilon_decay)
                episode_return = 0  # 初始化回合总回报
                episode_steps = 0  # 初始化回合步数
                state, _ = env.reset()  # 重置环境
                while True:
                    # 计算平滑的最大 Q 值，用于监控 Q 值趋势
                    max_q_value = agent.max_q_value_of_state(state) * 0.005 + max_q_value * 0.995
                    max_q_value_list.append(max_q_value)
                    action = agent.take_action(state)  # 选择动作（ε-greedy 策略）
                    next_state, reward, terminated, truncated, info = env.step(action)  # 执行动作
                    replay_buffer.push_transition((state, action, reward, next_state, terminated or truncated))  # 存储经验
                    episode_steps += 1  # 步数加 1

                    # 每 100 步打印详细日志
                    if episode_steps % 100 == 0:
                        print(f"Iteration {i}, Episode {i_episode + 1}, Step {episode_steps}: "
                              f"State = {state}, Action = {action}, Reward = {reward}, "
                              f"Next State = {next_state}, Distance = {info.get('distance_to_target', 'N/A')}, "
                              f"Epsilon = {agent.epsilon:.4f}, Max Q = {max_q_value:.4f}, "
                              f"State dtype = {state.dtype}")

                    # 当回放池足够大时，开始训练
                    if replay_buffer.len_buffer() > minimal_size:
                        samples = replay_buffer.sample(batch_size)  # 采样批量经验
                        states, actions, rewards, next_states, dones = zip(*samples)  # 解包经验
                        # 创建训练数据，确保数据类型为 np.float32
                        transition_dict = {
                            'states': np.array(states, dtype=np.float32),
                            'actions': np.array(actions, dtype=np.int64),
                            'next_states': np.array(next_states, dtype=np.float32),
                            'rewards': np.array(rewards, dtype=np.float32),
                            'dones': np.array(dones, dtype=np.float32)
                        }
                        agent.update(transition_dict)  # 更新 Q 网络

                    state = next_state  # 更新当前状态
                    episode_return += reward  # 累加回合回报

                    # 如果回合结束（终止或截断）
                    if terminated or truncated:
                        # 每 10 回合打印回合总结
                        if (i_episode + 1) % 10 == 0:
                            print(f"Iteration {i}, Episode {i_episode + 1}: "
                                  f"Return = {episode_return:.2f}, Steps = {episode_steps}, "
                                  f"Avg Return (last 10) = {np.mean(return_list[-10:]):.2f}, "
                                  f"Buffer Size = {replay_buffer.len_buffer()}, "
                                  f"Target Distance = {info.get('distance_to_target', 'N/A'):.2f}")
                            env.render()  # 渲染环境
                        break

                return_list.append(episode_return)  # 记录回合回报
                # 更新进度条显示
                if (i_episode + 1) % 10 == 0:
                    pbar.set_postfix({
                        'episode': '%d' % (num_episodes / 20 * i + i_episode + 1),
                        'return': '%.3f' % np.mean(return_list[-10:]),
                        'steps': '%d' % episode_steps
                    })
                pbar.update(1)

    # 绘制训练结果
    mv_return_list = moving_average(return_list, 29)  # 计算回报的移动平均
    episodes_list = list(range(len(return_list)))
    plt.figure(figsize=(12, 8))
    plt.plot(episodes_list, return_list, label='raw', alpha=0.5)  # 原始回报曲线
    plt.plot(episodes_list, mv_return_list, label='moving ave')  # 平滑回报曲线
    plt.xlabel('Episodes')
    plt.ylabel('Returns')
    plt.title('DQN on RollingBall')
    plt.legend()
    plt.savefig('./result/DQN.png')  # 保存回报曲线
    plt.show()

    # 绘制最大 Q 值曲线
    frames_list = list(range(len(max_q_value_list)))
    plt.plot(frames_list, max_q_value_list)
    plt.axhline(max(max_q_value_list), c='orange', ls='--')  # 标记最大 Q 值
    plt.xlabel('Frames')
    plt.ylabel('Max Q_value')
    plt.title('DQN on RollingBall')
    plt.savefig('./result/DQN_MaxQ.png')  # 保存 Q 值曲线
    plt.show()